# Datagrabbing Notebook

In [ ]:
#Bulk files:
#http://download.companieshouse.gov.uk/en_pscdata.html
#wget http://download.companieshouse.gov.uk/persons-with-significant-control-snapshot-2016-10-25.zip
#unzip http://download.companieshouse.gov.uk/persons-with-significant-control-snapshot-2016-10-25.zip

In [1]:
!ls

Companies House - Significant Control Snapshot - Loader.ipynb
docker-compose.yml
persons-with-significant-control-snapshot-2016-10-25.txt
persons-with-significant-control-snapshot-2016-10-25.zip
README.md


In [2]:
#Preview the data
!head -n 1 persons-with-significant-control-snapshot-2016-10-25.txt

{"company_number":"09145694","data":{"address":{"address_line_1":"Reading Road","locality":"Henley-On-Thames","postal_code":"RG9 1DP","premises":"161","region":"Oxfordshire"},"country_of_residence":"England","date_of_birth":{"month":2,"year":1977},"etag":"26281d9bedb2d102359f6afc3cb8cf62bb4a7f01","kind":"individual-person-with-significant-control","links":{"self":"/company/09145694/persons-with-significant-control/individual/bIhuKnMFctSnjrDjUG8n3NgOrlU"},"name":"Mrs Nga Thanh Wildman","name_elements":{"forename":"Nga","middle_name":"Thanh","surname":"Wildman","title":"Mrs"},"nationality":"Vietnamese","natures_of_control":["ownership-of-shares-50-to-75-percent"],"notified_on":"2016-04-06"}}


In [3]:
#How many records?
!wc -l persons-with-significant-control-snapshot-2016-10-25.txt

1270822 persons-with-significant-control-snapshot-2016-10-25.txt


So we have JSON data - Postgresql does support json now, but let's use mongo...

In [4]:
!pip3 install pymongo

    100% |████████████████████████████████| 337kB 2.2MB/s 


In [1]:
import pymongo

In [2]:
from pymongo import MongoClient
c=MongoClient('mongodb',27017)
c.database_names()

['ch', 'local']

In [3]:
#Create a db
db=c.ch

In [8]:
#Import the data - may take a bit of time...
import json
from itertools import islice

import codecs
loadsize=500
data=[]
db.sigcon.drop()
with codecs.open('persons-with-significant-control-snapshot-2016-10-25.txt', 'r', 'utf-8-sig') as f:
    for line in f:
        data.append(json.loads(line))
        #DO bulk intserts
        if len(data)>loadsize:
            db.sigcon.insert_many(data)
            data=[]
    if len(data)>0:db.sigcon.insert_many(data)

In [4]:
#Check we can query them
db.sigcon.find_one()

{'_id': ObjectId('580f756a28e8d3000a5061fb'),
 'company_number': '09145694',
 'data': {'address': {'address_line_1': 'Reading Road',
   'locality': 'Henley-On-Thames',
   'postal_code': 'RG9 1DP',
   'premises': '161',
   'region': 'Oxfordshire'},
  'country_of_residence': 'England',
  'date_of_birth': {'month': 2, 'year': 1977},
  'etag': '26281d9bedb2d102359f6afc3cb8cf62bb4a7f01',
  'kind': 'individual-person-with-significant-control',
  'links': {'self': '/company/09145694/persons-with-significant-control/individual/bIhuKnMFctSnjrDjUG8n3NgOrlU'},
  'name': 'Mrs Nga Thanh Wildman',
  'name_elements': {'forename': 'Nga',
   'middle_name': 'Thanh',
   'surname': 'Wildman',
   'title': 'Mrs'},
  'nationality': 'Vietnamese',
  'natures_of_control': ['ownership-of-shares-50-to-75-percent'],
  'notified_on': '2016-04-06'}}